# Tutorial 3: How simulations define your predictions
The inverse problem has no unique solution as it is ill-posed. In order to solve it we need to constraint the space of possible solutions. While inverse solutions like minimum-norm estimates have an explicit constraint of minimum-energy, the constraints with esinet are implicit and mostly shaped by the simulations.

This tutorial aims the relation between simulation parameters and predictions.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# import mne
# import numpy as np
# from copy import deepcopy
# import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info
from scipy.stats import pearsonr
from matplotlib import pyplot as plt
plot_params = dict(surface='white', hemi='both', verbose=0)

## Create Forward model
First we create a template forward model which comes with the esinet package

In [69]:
info = get_info(sfreq=100)
fwd = create_forward_model(sampling="ico4", info=info)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    2.0s remaining:    3.5s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    2.0s remaining:    1.2s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    2.1s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.2s remaining:    0.5s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.3s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:    0.5s remaining:    0.9s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    0.5s remaining:    0.3s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    0.6s finished


# Extent

## Simulate

In [18]:
n_samples = 10000
settings = dict(duration_of_trial=0., number_of_sources=1)
sim = Simulation(fwd, info, settings=settings).simulate(n_samples=n_samples)

Simulating data based on sparse patches.


100%|██████████| 10000/10000 [00:00<00:00, 27248.09it/s]


source data shape:  (1284, 1) (1284, 1)


100%|██████████| 10000/10000 [00:09<00:00, 1075.47it/s]


## Create Data

In [22]:
import numpy as np
X = np.squeeze(np.stack([eeg.average().data for eeg in sim.eeg_data]))
X = np.stack([(x - np.mean(x)) / np.std(x) for x in X], axis=0)
y = np.array([extent[0] for extent in sim.simulation_info.extents.values])
y

array([11.40710349,  2.22236401, 11.1196467 , ..., 10.09542751,
       34.24898695,  2.2508002 ])

## Build and Train

In [42]:
import tensorflow as tf
from tensorflow.keras.layers import Dense

leadfield, pos = util.unpack_fwd(fwd)[1:3]
n_channels, n_dipoles = leadfield.shape
input_shape = (None, None, n_channels)
tf.keras.backend.set_image_data_format('channels_last')

n_dense_units = 300
activation_function = "tanh"
batch_size = 32
epochs = 100

model = tf.keras.Sequential()
model.add(Dense(units=n_dense_units, activation=activation_function))
# model.add(Dense(units=n_dense_units, activation=activation_function))
# model.add(Dense(units=n_dense_units, activation=activation_function))

# Add output layer
model.add(Dense(1, activation='linear'))

# Build model with input layer
model.build(input_shape=input_shape)

model.compile(loss='mean_squared_error', optimizer="adam")
model.summary()

model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.15)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, None, 300)         18600     
_________________________________________________________________
dense_17 (Dense)             (None, None, 1)           301       
Total params: 18,901
Trainable params: 18,901
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
266/266 [==============================] - 1s 2ms/step - loss: 577.3128 - val_loss: 237.5743
Epoch 2/100
266/266 [==============================] - 0s 1ms/step - loss: 205.0219 - val_loss: 214.0326
Epoch 3/100
266/266 [==============================] - 0s 1ms/step - loss: 196.0475 - val_loss: 200.2537
Epoch 4/100
266/266 [==============================] - 0s 1ms/step - loss: 192.2560 - val_loss: 199.5821
Epoch 5/100
266/266 [==============================] - 0s 1ms/step - loss: 190.9534 - val_lo

## Evaluate

In [41]:
n_samples = 1000
settings = dict(duration_of_trial=0., number_of_sources=1)
sim_test = Simulation(fwd, info, settings=settings).simulate(n_samples=n_samples)

X_test = np.squeeze(np.stack([eeg.average().data for eeg in sim_test.eeg_data]))
X_test = np.stack([(x - np.mean(x)) / np.std(x) for x in X_test], axis=0)
y_test = np.array([extent[0] for extent in sim_test.simulation_info.extents.values])

y_pred = model.predict(X_test)[:, 0]
%matplotlib qt
plt.figure()
plt.scatter(y_test, y_pred)
plt.xlabel("True")
plt.ylabel("Predicted")
plt.ylim(0,51)
plt.xlim(0,51)
r, _ = pearsonr(y_test, y_pred)
plt.title(f"r={r}")

Simulating data based on sparse patches.


100%|██████████| 1000/1000 [00:00<00:00, 34456.36it/s]


source data shape:  (1284, 1) (1284, 1)


100%|██████████| 1000/1000 [00:00<00:00, 1129.93it/s]


Text(0.5, 1.0, 'r=0.564959964322163')

# Position

## sim

In [70]:
n_samples = 10000
settings = dict(duration_of_trial=0., number_of_sources=1)
sim = Simulation(fwd, info, settings=settings).simulate(n_samples=n_samples)

Simulating data based on sparse patches.


100%|██████████| 10000/10000 [00:00<00:00, 15674.19it/s]


source data shape:  (5124, 1) (5124, 1)


100%|██████████| 10000/10000 [00:09<00:00, 1054.64it/s]


# create data

In [71]:
import numpy as np
X = np.squeeze(np.stack([eeg.average().data for eeg in sim.eeg_data]))
X = np.stack([(x - np.mean(x)) / np.std(x) for x in X], axis=0)
y = np.array([extent[0] for extent in sim.simulation_info.positions.values])

## create and train

In [72]:
import tensorflow as tf
from tensorflow.keras.layers import Dense

leadfield, pos = util.unpack_fwd(fwd)[1:3]
n_channels, n_dipoles = leadfield.shape
input_shape = (None, None, n_channels)
tf.keras.backend.set_image_data_format('channels_last')

n_dense_units = 200
activation_function = "tanh"
batch_size = 32
epochs = 100

model = tf.keras.Sequential()
model.add(Dense(units=n_dense_units, activation=activation_function))
model.add(Dense(units=n_dense_units, activation=activation_function))
model.add(Dense(units=n_dense_units, activation=activation_function))

# Add output layer
model.add(Dense(3, activation='linear'))

# Build model with input layer
model.build(input_shape=input_shape)

model.compile(loss='mean_squared_error', optimizer="adam")
model.summary()

model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.15)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, None, 200)         12400     
_________________________________________________________________
dense_28 (Dense)             (None, None, 200)         40200     
_________________________________________________________________
dense_29 (Dense)             (None, None, 200)         40200     
_________________________________________________________________
dense_30 (Dense)             (None, None, 3)           603       
Total params: 93,403
Trainable params: 93,403
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
266/266 [==============================] - 1s 4ms/step - loss: 969.7183 - val_loss: 692.4044
Epoch 2/100
266/266 [==============================] - 1s 5ms/step - loss: 503.6608 - val_loss: 296.3875
Epoch 3/100
266/266 [================

KeyboardInterrupt: 

## eval

In [88]:
n_samples = 100
settings = dict(duration_of_trial=0., number_of_sources=1)
sim_test = Simulation(fwd, info, settings=settings).simulate(n_samples=n_samples)

X_test = np.squeeze(np.stack([eeg.average().data for eeg in sim_test.eeg_data]))
X_test = np.stack([(x - np.mean(x)) / np.std(x) for x in X_test], axis=0)
y_test = np.array([extent[0] for extent in sim_test.simulation_info.positions.values])

y_pred = model.predict(X_test)
distances = np.sqrt(((y_pred - y_test)**2).sum(axis=1))
plt.figure()
plt.hist(distances)
plt.xlabel("Euclidean distance to true maximum")
plt.title(f"median={np.median(distances)}")

stc = sim_test.source_data[0]
brain = stc.plot(**plot_params)
brain.add_foci(y_pred[0, :], coords_as_verts=False, hemi="lh", color="red")
brain.add_foci(y_pred[0, :], coords_as_verts=False, hemi="rh", color="red")



Simulating data based on sparse patches.


100%|██████████| 100/100 [00:00<00:00, 16699.06it/s]


source data shape:  (5124, 1) (5124, 1)


100%|██████████| 100/100 [00:00<00:00, 1063.52it/s]


In [89]:
distances

array([19.31613181,  2.04705525, 14.1597461 ,  5.27064398,  1.24725668,
        4.36241445, 17.47596471,  6.62718545, 13.15538461,  7.84819766,
        8.61954005,  6.39945473, 14.76710841,  3.26612865,  9.54420082,
       10.87083289,  6.81587002,  6.41546564, 10.13584463, 10.49176123,
        9.67520817,  2.31887653, 15.48955206,  3.6202502 ,  6.84626395,
        1.67094146,  7.69778097, 22.35361877,  8.36259844,  3.94436435,
        5.74545302,  4.08565251,  6.28159072,  1.52095045,  8.10723901,
       30.65317859,  9.1726696 ,  9.52249766,  4.59374964,  8.80919645,
        4.39782712,  4.68890662, 12.64326095,  1.84111413, 11.10927796,
        4.1710168 ,  8.98015923,  7.91337976,  2.71961383, 19.25576127,
       10.95768192, 19.59450877,  7.20493244,  4.57305047, 13.62665406,
        4.09885352,  7.0607517 ,  5.96046843,  2.61638712, 15.2390202 ,
       15.50284665,  5.4391461 ,  9.02056581,  5.72645869, 13.97868068,
        5.93857676, 10.61861199,  4.86034232,  7.82606373,  2.76